# Google Drive API

**Download credentials.json from**
https://developers.google.com/drive/api/v3/quickstart/python

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(pageSize=1000, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

# if not items:
#     print('No files found.')
# else:
#     print('Files:')
#     for item in items:
#         print(u'{0} ({1})'.format(item['name'], item['id']))



In [ ]:
items[:5]

In [ ]:
# Batch delete
from tqdm.notebook import tqdm

def delete_file(request_id, response, exception):
  if exception is not None:
    print(response, exception)
  else:
    pass

batch = service.new_batch_http_request(callback=delete_file)


for i, file in tqdm(enumerate(items)):
    name = file["name"]
    idn = file["id"]
    if "Train_" in name:
        batch.add(service.files().delete(fileId=idn))
    
    if i%10 == 0:
        batch.execute()
        batch = service.new_batch_http_request(callback=delete_file)
        
batch.execute()

#         service.files().delete(fileId=idn).execute()
# service.files().delete(fileId="1360Xx6d5yc8h8QE2FcnNPpzYD8pKf1SY")